# Time series
Time Series sequences and prediction of sizonal variations and trends in data by [Laurence Moroney with examples](https://github.com/lmoroney/dlaicourse/tree/master/TensorFlow%20In%20Practice/Course%204%20-%20S%2BP).

Time Series :
- **Univariate** - single value at each step in time
- **Multivariate** - multiple values at each step in time

Used as search of patterns in time:
- Prediction of future = forecast, past = imputation
- Anomaly detection
- Feature detection in speach recognition

Common patterns in time series:
- **trend** constant or linear function
- **seasonality** when same patterns repeat at predictable intervals
- **combination** of trend and seasonality
- **autocorrelation** it correlates with a delayed copy of itself often called a lag.
- **white noise** truly random signal can't be predicted.
- **Non-stationary** different patterns within different time scope in data.

Note **Non-stationary** time series may need few different models (each with limited time scope in data) + anomaly detection.

![autocorrelation1](autocorrelation1.jpg)
![autocorrelation2](autocorrelation2.jpg)
![realmix](realmix.jpg)
![non-stationary](non-stationary.jpg)



## Metric in timeseries
train, test data sets and error measure in TS


### Fixed Partitions
Note difference TS-ML with classical ML:
- You whant to include **full cycles** in all sets to avoid bias of diferrent representation in data sets.
- tune hyper parameters on validation set
- include validation set in training and check final performance on test set
- if fianl performance is good before releasing, train production model on whole data set including test, as test has most recent data and more relevant signal for the forecast.

![fixed-partitions.jpg](fixed-partitions.jpg)

### Rolling Partitions

- take small portion of training period and try to predict next day (still inside training set)
- add that day to training and try to predict next day
- repeat until all training set is used. 
Monitor model performance on validation set. Model accuracy should gradualy increase.

Sometime you don't allocate test set and use next day life data as your test set.

![roll-partitions.jpg](roll-partitions.jpg)


### Metrics for evaluating performance

L2 metric (MSE) penalizes proportinal to error value, big errors affects function more severly.

L1 metric (MAE) does not penalize large errors as much as the mse does. 
If your gain or your loss is just slightly proportional to the size of the error, then the mae may be better.

MAPE - mean absolute percentage error or mape, this is the mean ratio between the absolute error and the absolute value, this gives an idea of the size of the errors compared to the values.

```python
mae = keras.metrics.mean_absolute_error(y_validation, prediction).numpy()
```

![](err-metric.jpg)

Base line (relative performance) - naive forecast, moving average (rolling average), moving average of difference.

naive forecast - f(t+1) = f(t)

![](relative-performance.jpg)

Note, 
mae(naive forecast) = 5.7
mae(moving average) = 7.14
,  cause components: trend, seasonality (?). **differencing** is a technique to remove trend, seasonality. So instead of studying the time series itself, we study the difference between the value at time T and the value at an earlier period. Differencing lag.

![](differencing1.jpg)

base line prediction `f(t) = f(t-lag) + roll_avg(differencing[t, window])`
with moving window = 32

![](differencing2.jpg)

smoth by average 2 with centered window = \[t-360, t+5 \]?! for prediction (validation test?) we can't use centered window as we dont know future, use trailing windows.

![](differencing3.jpg)

Note, 
mae(naive forecast) = 5.7
mae(smooth moving average) = 4.5!

statistical forecasting = use linear aproximations in TS (mainly smooth moving average)
